## A Variational Classifier for Heart Disease

This project is based on the PennyLane variational classifier tutorial found [here](https://pennylane.ai/qml/demos/tutorial_variational_classifier.html). To shake things up and to not repeat the solutions presented in the tutorial, the Iris dataset will not be used. Instead, a classifier will be produced that predicts the presense of heart disease based on a set of key indicators.

### Dataset

The dataset was originally taken from the annual CDC health status survey from 2020, but the current version was preprocessed by [Kamil Pytlak](https://github.com/kamilpytlak) and posted on [Kaggle](https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease). If you find this dataset interesting and would like to use it in one of your own projects, please provide appropriate attribution to the CDC surveyors and Kamil for their work!

### Dataset Preprocessing

First, we begin by loading the dataset from a CSV via Pandas and taking a look at the features available to us.

In [6]:
import pandas

data = pandas.read_csv("./data/heart_2020_cleaned.csv")
first_row = data.iloc[0]
num_features = len(first_row) - 1

print(first_row)
print(f"Number of Features: {num_features}")

HeartDisease               No
BMI                      16.6
Smoking                   Yes
AlcoholDrinking            No
Stroke                     No
PhysicalHealth            3.0
MentalHealth             30.0
DiffWalking                No
Sex                    Female
AgeCategory             55-59
Race                    White
Diabetic                  Yes
PhysicalActivity          Yes
GenHealth           Very good
SleepTime                 5.0
Asthma                    Yes
KidneyDisease              No
SkinCancer                Yes
Name: 0, dtype: object
Number of Features: 17


There are a total of 18 features with 14 of them being categorical and each category represented as a string. Since the classifier will only take in numerical values, we need to map our categories to real numbers.

In [7]:
categorical_columns = [
    "HeartDisease",
    "Smoking",
    "AlcoholDrinking",
    "Stroke",
    "DiffWalking",
    "Sex",
    "AgeCategory",
    "Race",
    "Diabetic",
    "PhysicalActivity",
    "GenHealth",
    "Asthma",
    "KidneyDisease",
    "SkinCancer",
]

for column in categorical_columns:
    data[column] = data[column].astype("category").cat.codes
    
first_row = data.iloc[0]
print(first_row)

HeartDisease         0.0
BMI                 16.6
Smoking              1.0
AlcoholDrinking      0.0
Stroke               0.0
PhysicalHealth       3.0
MentalHealth        30.0
DiffWalking          0.0
Sex                  0.0
AgeCategory          7.0
Race                 5.0
Diabetic             2.0
PhysicalActivity     1.0
GenHealth            4.0
SleepTime            5.0
Asthma               1.0
KidneyDisease        0.0
SkinCancer           1.0
Name: 0, dtype: float64


Now that our feature values are numerical, we will randomly select a subset of the dataset to reduce training time and then split that subset into training, validation, and testing sets.

In [8]:
data_subset_size = 1000
training_set_fraction = 0.8
validation_set_fraction = 0.1
testing_set_fraction = 0.1

training_end = int(data_subset_size * training_set_fraction)
validation_end = int(data_subset_size * (training_set_fraction + validation_set_fraction))

data = data.sample(n=data_subset_size)

training_set = data.iloc[:training_end]
validation_set = data.iloc[training_end:validation_end]
testing_set = data.iloc[validation_end:]

print(f"Data Subset Size: {data_subset_size}")
print(f"Training Set Size: {len(training_set)}")
print(f"Validation Set Size: {len(validation_set)}")
print(f"Testing Set Size: {len(testing_set)}")

Data Subset Size: 1000
Training Set Size: 800
Validation Set Size: 100
Testing Set Size: 100


Before proceeding, we split the training, validation, and testing sets into labels and features. We also convert the labels from values in the set `{0, 1}` to values in the set `{-1, 1}`. Expected value measurements from our quantum circuit will be within the interval `[-1, 1]` and having our label values as the endpoints of this interval makes computing the cost function easier.

In [9]:
from pennylane import numpy as np

label_column = "HeartDisease"

training_labels = np.array(training_set[label_column].values, dtype=np.float64, requires_grad=True)
validation_labels = np.array(validation_set[label_column].values, dtype=np.float64, requires_grad=True)
testing_labels = np.array(testing_set[label_column].values, dtype=np.float64, requires_grad=True)

training_features = np.array(training_set.drop(label_column, axis=1).values, requires_grad=True)
validation_features = np.array(validation_set.drop(label_column, axis=1).values, requires_grad=True)
testing_features = np.array(testing_set.drop(label_column, axis=1).values, requires_grad=True)

training_labels[np.where(training_labels == 0)] = -1
validation_labels[np.where(validation_labels == 0)] = -1
testing_labels[np.where(testing_labels == 0)] = -1

### Data Encoding

Before using our dataset, we need to consider how to encode our feature vectors into quantum states. There are a number of ways of going about state preparation, but which is best for our use case?

Below is a table taken from [Schuld and Petruccione (2018)](https://link.springer.com/book/10.1007/978-3-319-96424-9) which contains four encoding strategies for a dataset of M inputs with N features each:

| Encoding| Number of Qubits | Runtime of State Prep | Input Features |
|:-:|:-:|:-:|:-:|
| Basis | N | O(MN) | Binary |
| Amplitude | log(N) | O(MN)/O(log(MN))\* | Continuous |
| Qsample | N | O(2^N)/O(N)\* | Binary |
| Hamiltonian | log(N) | O(MN)/O(log(MN))\*| Continuous|

Note that the authors indicate that certain datasets or models can be encoded in the runtimes which have asterixes next to them. Looking at the table, we can immediately eliminate the basis and Qsample encoding strategies as they are only applicable to binary features. Although some of our features are binary, a great deal of them are continuous.

Amplitude encoding and Hamiltonian encoding have the same orders for their space and time complexities, so if we were solely concerned with the number of qubits or runtime, either should work fine. We will use PennyLane's implementation of amplitude encoding within this project.


### Variational Circuit Construction

Now that we've preprocessed the dataset and decided on a data encoding strategy, the variational circuit can be constructed. We begin with importing the PennyLane package, the standard Python mathematics package, and NumPy. Our variational circuit will require a classical optimization strategy, so we also import PennyLane's implementation of the Nesterov momentum optimizer.

In [10]:
import pennylane as qml

from pennylane import math
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

Next we define a series of constants which will define the structure of the circuit. These are the number of features, the number of layers, and the number of qubits (or wires in the parlance of PennyLane). Note that the feature count is determined from the dataset and that the number of layers is directly configurable. The number of qubits is determined by the rule that amplitude encoding requires $n$ qubits to encode $2^n$ features.

In [11]:
num_features = training_features.shape[1]
num_layers = 2

"""
Amplitude encoding requires n qubits to encode 2^n features.
Here we apply a base two logarithm via the base change formula to determine n.
"""
num_wires = int(math.ceil(math.log(num_features) / math.log(2)))

Knowing the number of qubits enables us to define a function for state preparation. The function uses the `AmplitudeEmbedding` function to amplitude encode the provided features within the circuit's qubits.

In [12]:
def state_preparation(features):
    """
    Prepares a state by amplitude encoding the provided features.
    
    Arguments:
    
    features - An instance of features from the dataset.
    """
    
    # Normalizing features and encoding them with padding.
    qml.AmplitudeEmbedding(
        features=features,
        wires=range(num_wires),
        pad_with=0.,
        normalize=True,
    )

At this point, data can be encoded into the circuit, but it is not operated upon. A variational circuit consists of a series of layers which carry out operations on quantum states based on a set of parameters. In machine learning, these parameters are known as weights. There are different ways to construct these layers, but here we take the approach of each set of three weights defining an arbitrary rotation applied to a qubit. Following the rotation, each qubit is entangled with one of its neighbors until there is a chain of entanglement passing through all qubits in a given layer.

In [13]:
def layer(layer_weights):
    """
    Adds a layer to the variational circuit.
    
    Arguments:
    
    weights - A matrix of weights which define the phi, theta,
    and omega values for the circuit rotations.
    """
    num_wires = layer_weights.shape[0]
    
    for wire_index, wire_weights in enumerate(layer_weights):
        # Applying rotation on qubit.
        qml.Rot(*list(wire_weights), wires=wire_index)
        
        # Entangling qubit with its neighbor, wrapping around at the end.
        qml.CNOT(wires=[wire_index, (wire_index + 1) % num_wires])

A quantum device is initialized and the state preparation and layering elements are combined to produce a variational circuit. The circuit is executed and repeated measurements are taken in the Z measurement basis to determine the circuit's expected value for the given input. The expected value is treated as the circuit's response to the provided instance of our classification problem. 

In [14]:
dev = qml.device('default.qubit', wires=num_wires)

@qml.qnode(dev)
def circuit(features, weights):
    state_preparation(features)
    
    for layer_weights in weights:
        layer(layer_weights)

    return qml.expval(qml.PauliZ(0))

### Training a Variational Classifier

A variational classifier can be created by calling the variational circuit with weights, a bias term, and a set of features to classify. The classifier is unlikely to do very well unless it has been trained to optimize its weights and bias. Below are functions for creating a classifier, returning its predictions, and evaluating those predictions in terms of loss. Loss is determined by the average squared difference between the true labels corresponding to a set of features and the classifier's predicted label for those features.

In [15]:
def variational_classifier(weights, bias, features):
    return circuit(features, weights) + bias

def square_loss(labels, predictions):
    return np.sum((labels - predictions) ** 2) / len(labels)

def accuracy(labels, predictions):
    num_correct = len(np.where(predictions == labels)[0])
    return num_correct / len(labels)

def predict(weights, bias, features):
    predictions = np.array([np.sign(variational_classifier(weights, bias, f)) for f in features])
    return predictions

def cost(weights, bias, features, labels):
    predictions = np.array([variational_classifier(weights, bias, f) for f in features])
    return square_loss(labels, predictions)

The training process begins with the initialization of the optimizer, setting the batch size, and setting the number of training iterations, known as epochs. The batch size is the number of instances which are randomly selected and classified during a given epoch. For example, if the batch size was `5`, then `5` instances would be classified per epoch.

Next, a set of starting weights is randomly initialized and the bias term is set to zero.

In [16]:
optimizer = NesterovMomentumOptimizer()
batch_size = 50
num_epochs = 30

# A rotation is applied to every qubit and there are three parameters
# for each rotation: phi, theta, and omega.
weights = np.random.rand(num_layers, num_wires, 3, requires_grad=True)

bias = np.array(0.0, requires_grad=True)

The core of the training process consists of randomly selecting batches of features, making predictions using those features, and then adjusting the weights and bias based on the classifiers performance. As the classifier goes through training iterations, the cost associated with its predictions declines, which results in a corresponding increase in accuracy on both the training and validation sets.

In [17]:
for epoch in range(num_epochs):
    # Selecting batch features and labels.
    batch_index = np.random.randint(0, len(training_features), (batch_size,))
    training_batch_features = training_features[batch_index]
    training_batch_labels = training_labels[batch_index]
    
    # Updating parameters using optimizer.   
    weights, bias, _, _ = optimizer.step(
        cost,
        weights,
        bias,
        training_batch_features,
        training_batch_labels,
    )
    
    # Predict labels on training and validation sets.
    training_predictions = predict(weights, bias, training_features)
    validation_predictions = predict(weights, bias, validation_features)
    
    # Computing accuracy on training and validation sets.
    training_accuracy = accuracy(training_labels, training_predictions)
    validation_accuracy = accuracy(validation_labels, validation_predictions)
    
    print(
        "Epoch: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(epoch + 1, cost(
            weights,
            bias,
            training_features,
            training_labels,
        ), training_accuracy, validation_accuracy)
    )

testing_predictions = predict(weights, bias, testing_features)
testing_accuracy = accuracy(testing_labels, testing_predictions)
print(f"Testing Accuracy: {testing_accuracy}")

Epoch:     1 | Cost: 1.5534128 | Acc train: 0.1462500 | Acc validation: 0.0900000 
Epoch:     2 | Cost: 1.4097524 | Acc train: 0.1575000 | Acc validation: 0.1000000 
Epoch:     3 | Cost: 1.2267578 | Acc train: 0.1750000 | Acc validation: 0.1400000 
Epoch:     4 | Cost: 1.0355481 | Acc train: 0.3175000 | Acc validation: 0.3100000 
Epoch:     5 | Cost: 0.8494911 | Acc train: 0.8862500 | Acc validation: 0.9300000 
Epoch:     6 | Cost: 0.6852686 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:     7 | Cost: 0.5553814 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:     8 | Cost: 0.4596633 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:     9 | Cost: 0.3970745 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    10 | Cost: 0.3623222 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    11 | Cost: 0.3516563 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    12 | Cost: 0.3559749 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoc

After the model has been trained, the weights and bias term can be saved off for later use.

In [18]:
model_weights_filepath = "models/heart_disease_weights.npy"
model_bias_filepath = "models/heart_disease_bias.npy"

np.save(model_weights_filepath, weights)
np.save(model_bias_filepath, bias)

### Results

In my experimentation, I was able to produce a model that has 90.6% accuracy on the training set, 93% accuracy on the validation set, and 97% accuracy on the test set. Assuming the test set is a decent generalization of the population, this means that the classifier will correctly identify heart disease for approximately 9 out of 10 individuals who answer the survey questions truthfully.

The output from the best experiment is provided below:

```
Epoch:     1 | Cost: 1.5534128 | Acc train: 0.1462500 | Acc validation: 0.0900000 
Epoch:     2 | Cost: 1.4097524 | Acc train: 0.1575000 | Acc validation: 0.1000000 
Epoch:     3 | Cost: 1.2267578 | Acc train: 0.1750000 | Acc validation: 0.1400000 
Epoch:     4 | Cost: 1.0355481 | Acc train: 0.3175000 | Acc validation: 0.3100000 
Epoch:     5 | Cost: 0.8494911 | Acc train: 0.8862500 | Acc validation: 0.9300000 
Epoch:     6 | Cost: 0.6852686 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:     7 | Cost: 0.5553814 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:     8 | Cost: 0.4596633 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:     9 | Cost: 0.3970745 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    10 | Cost: 0.3623222 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    11 | Cost: 0.3516563 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    12 | Cost: 0.3559749 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    13 | Cost: 0.3696567 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    14 | Cost: 0.3878293 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    15 | Cost: 0.4066116 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    16 | Cost: 0.4230051 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    17 | Cost: 0.4354312 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    18 | Cost: 0.4421084 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    19 | Cost: 0.4447286 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    20 | Cost: 0.4425996 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    21 | Cost: 0.4344603 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    22 | Cost: 0.4232259 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    23 | Cost: 0.4103675 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    24 | Cost: 0.3961825 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    25 | Cost: 0.3815703 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    26 | Cost: 0.3700311 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    27 | Cost: 0.3602819 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    28 | Cost: 0.3528935 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    29 | Cost: 0.3479278 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Epoch:    30 | Cost: 0.3449911 | Acc train: 0.9062500 | Acc validation: 0.9300000 
Testing Accuracy: 0.97
```

The model weights and bias values are located [here](models/heart_disease_weights_final.npy) and [here](models/heart_disease_bias_final.npy) respectively.